In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

%pip install semantic-link
%pip install semantic-link-labs

StatementMeta(, 83814574-14d7-4a20-b932-b86bc51200c6, 9, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 50.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 131.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 119.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 114.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 207.6 MB/s eta 0:00:00
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.8.0
    Not uninstalling semantic-link-sempy at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-

In [2]:
import sempy.fabric as fabric
import sempy_labs as sempy_labs
from sempy_labs import migration, report, directlake
from sempy_labs import lakehouse as lake
from sempy_labs.tom import connect_semantic_model
from pyspark.sql.functions import current_timestamp
import pandas as pd

StatementMeta(, 83814574-14d7-4a20-b932-b86bc51200c6, 11, Finished, Available, Finished)

In [3]:
# clear

spark.sql("DELETE from item")
spark.sql("DELETE from reportpages")
spark.sql("DELETE from reportpageviews")
spark.sql("DELETE from report")
spark.sql("DELETE from workspace")

StatementMeta(, 83814574-14d7-4a20-b932-b86bc51200c6, 12, Finished, Available, Finished)

DataFrame[num_affected_rows: bigint]

In [4]:
# Define Lakehouse name and description. This will the LH where all documentation will be saved
LH_Name = "LH_CatManUsage"
LH_desc = "Lakehouse for CatMan Usage and Items"

# Get current workspace details
current_workspace_id = fabric.get_workspace_id()
current_workspace_name = fabric.resolve_workspace_name(current_workspace_id)
print(f'Current workspace ID: {current_workspace_id}')
print(f'Current workspace name: {current_workspace_name}')

# Check if the Lakehouse already exists, if not, create it
# List existing lakehouses and check if the specified one already exists
lakehouse_list = sempy_labs.list_lakehouses()

if LH_Name in lakehouse_list['Lakehouse Name'].values:
    print("Lakehouse already exists")
else:
    # Create a new Lakehouse
    mssparkutils.lakehouse.create(name=LH_Name, description=LH_desc, workspaceId=current_workspace_id)
    print("Lakehouse created successfully")

# Mount the Lakehouse for direct file system access
lakehouse = mssparkutils.lakehouse.get(LH_Name)
mssparkutils.fs.mount(lakehouse.get("properties").get("abfsPath"), f"/{LH_Name}")

# Retrieve and store local and ABFS paths of the mounted Lakehouse
local_path = mssparkutils.fs.getMountPath(f"/{LH_Name}")
lh_abfs_path = lakehouse.get("properties").get("abfsPath")
print(f'Local path: {local_path}')
print(f'ABFS path: {lh_abfs_path}')

StatementMeta(, 83814574-14d7-4a20-b932-b86bc51200c6, 13, Finished, Available, Finished)

Current workspace ID: 97588196-ee93-4a7f-9d84-e44a68725087
Current workspace name: CatMan Usage
Lakehouse already exists


SynapseWidget(Synapse.DataFrame, 92c220c1-03eb-4c6a-a30b-78ea7aceb6cc)

Local path: /synfs/notebook/83814574-14d7-4a20-b932-b86bc51200c6/LH_CatManUsage
ABFS path: abfss://97588196-ee93-4a7f-9d84-e44a68725087@onelake.dfs.fabric.microsoft.com/2c4b8f66-d5f6-4fca-9c0b-d62c716a3eb7


In [5]:
# Initialize the list of workspaces if it's empty populate with current workspace ID
list_of_workspaces = fabric.list_workspaces()

print("Number of workspaces: ", len(list_of_workspaces))

# Transform column names for LH compatibility
list_of_workspaces.columns = list_of_workspaces.columns.str.replace('[^a-zA-Z0-9]', '', regex=True)
list_of_workspaces.columns = list_of_workspaces.columns.str.replace('[ ]', '', regex=True)

df_workspaces = spark.createDataFrame(list_of_workspaces)

# filter to only workspace as admin monitoring will have another type

df_workspaces = df_workspaces.filter(df_workspaces["Type"] == 'Workspace')

Table_Name = "Workspace"

df_workspaces.write.format("delta").option("mergeSchema", "true").mode("append").saveAsTable(Table_Name)



StatementMeta(, 83814574-14d7-4a20-b932-b86bc51200c6, 14, Finished, Available, Finished)

Number of workspaces:  429


In [ ]:

from delta.tables import DeltaTable
import pandas as pd
from pyspark.sql.functions import lit, current_timestamp


# Extract the 'Id' column from the DataFrame
workspace_ids = [row['Id'] for row in df_workspaces.select('Id').collect()]

# Iterate through each workspace
for workspace_id in workspace_ids:
    # Retrieve all items from the current fabric workspace
    Fabric_all_items = fabric.list_items(workspace=workspace_id)
    
    # Transform column names for LH compatibility
    Fabric_all_items.columns = Fabric_all_items.columns.str.replace('[^a-zA-Z0-9]', '', regex=True)
    Fabric_all_items.columns = Fabric_all_items.columns.str.replace('[ ]', '', regex=True)
    
    # Create a Spark DataFrame from the fabric items and save it to the Lakehouse
    try:
        sparkdf = spark.createDataFrame(Fabric_all_items)
        sparkdf = sparkdf.withColumn("LoadDate", current_timestamp())
        Table_Name = "item"
        sparkdf.write.format("delta").option("mergeSchema", "true").mode("append").saveAsTable(Table_Name)
    except Exception as e:
        print("error happened with workspace id:" + workspace_id)
        continue


In [ ]:

item_table = spark.read.table("item")

# Filter for items with the specified conditions
filtered_items = item_table.filter((item_table["DisplayName"] == "Usage Metrics Report") & (item_table["Type"] == "SemanticModel"))

# Collect the filtered rows for iteration
rows = filtered_items.collect()

for row in rows:
    try:
        dataset_id = row["Id"]
        workspace_id = row["WorkspaceId"]

        print("datasetid = " + dataset_id + " workspace id = " + workspace_id ) 

        df_table = fabric.read_table(dataset_id, table="Report page views", workspace=workspace_id)
    
        df_table.to_lakehouse_table("reportpageviews", mode='append')

        df_table_pages = fabric.read_table(dataset_id, table="Report pages", workspace=workspace_id)

        df_table_pages.to_lakehouse_table("reportpages", mode='append')

        df_table_reports = fabric.read_table(dataset_id, table="Reports", workspace=workspace_id)

        df_table_reports.to_lakehouse_table("report", mode='append')
        
    except Exception as e:
        print("error happened with workspace id:" + workspace_id)
        continue


StatementMeta(, , , Waiting, , Waiting)

StatementMeta(, a049b495-a76e-450d-979d-09b1c1ad5a45, 12, Finished, Available, Finished)

NameError: name 'Date' is not defined